# **PDF and CDF**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio
import seaborn as sns
import pandas as pd

In [ ]:
file_path = 'path/to/your/image.tif'
im_mask_path = 'path/to/your/im_mask.tif'
pm_mask_path = 'path/to/your/pm_mask.tif

In [ ]:
image = iio.imread(file_path)
im_mask = iio.imread(im_mask_path).astype(np.float32)
pm_mask = iio.imread(pm_mask_path).astype(np.float32)

In [ ]:
z = 1
frame_2_channel_0 = image[0, z, :, :]
frame_2_channel_1 = image[1, z, :, :]

In [ ]:
im_values = frame_2_channel_0[im_mask > 0] / frame_2_channel_1[im_mask > 0]
pm_values = frame_2_channel_0[pm_mask > 0] / frame_2_channel_1[pm_mask > 0]

In [ ]:
im_sorted = np.sort(im_values)
im_cdf = np.arange(1, len(im_sorted) + 1) / len(im_sorted) if len(im_sorted) > 0 else np.array([])

pm_sorted = np.sort(pm_values)
pm_cdf = np.arange(1, len(pm_sorted) + 1) / len(pm_sorted) if len(pm_sorted) > 0 else np.array([])

max_len = max(len(im_sorted), len(pm_sorted))

im_sorted = np.pad(im_sorted, (0, max_len - len(im_sorted)), constant_values=np.nan)
im_cdf = np.pad(im_cdf, (0, max_len - len(im_cdf)), constant_values=np.nan)

pm_sorted = np.pad(pm_sorted, (0, max_len - len(pm_sorted)), constant_values=np.nan)
pm_cdf = np.pad(pm_cdf, (0, max_len - len(pm_cdf)), constant_values=np.nan)

In [ ]:
cdf_data = pd.DataFrame({
    "IM_Value": im_sorted,
    "IM_CDF": im_cdf,
    "PM_Value": pm_sorted,
    "PM_CDF": pm_cdf
})

cdf_data.to_csv("cdf_results.csv", index=False)

## **Visualisation**

In [ ]:
im_color = "#00274d"
pm_color = "#ff4500"

fig, ax = plt.subplots(1, 2, figsize=(22, 10))

### PDF

In [ ]:
ax[0].hist(im_values, bins=50, density=True, color=im_color, alpha=0.5, linewidth=2)
ax[0].hist(pm_values, bins=50, density=True, color=pm_color, alpha=0.5, linewidth=3)
ax[0].set_xlabel("I(CFP/YFP)", fontsize=35, fontweight='bold', color="black")
ax[0].set_ylabel("Probability Density", fontsize=35, fontweight='bold', color="black")
ax[0].legend(["IM", "PM"], fontsize=30, loc="upper right", frameon=False)
ax[0].tick_params(axis='both', which='major', labelsize=35)

sns.kdeplot(im_values, label="IM ECFP/EYFP", ax=ax[0], linewidth=6, color=im_color)
sns.kdeplot(pm_values, label="PM ECFP/EYFP", ax=ax[0], linewidth=6, color=pm_color)

## CDF

In [ ]:
ax[1].axhline(0.5, color='black', linestyle='--', linewidth=4)
yticks = list(ax[1].get_yticks()) + [0.5]
ax[1].set_yticks(sorted(set(yticks)))
ax[1].set_xlabel("I(CFP/YFP)", fontsize=35, fontweight='bold', color="black")
ax[1].set_ylabel("Cumulative Probability", fontsize=35, fontweight='bold', color="black")
ax[1].legend(fontsize=30, loc="lower right", frameon=False)
ax[1].tick_params(axis='both', which='major', labelsize=35)
for spine in ax[0].spines.values():
    spine.set_linewidth(4)
for spine in ax[1].spines.values():
    spine.set_linewidth(4)

sns.ecdfplot(im_values, label="IM", ax=ax[1], linewidth=6, color=im_color, linestyle='solid')
sns.ecdfplot(pm_values, label="PM", ax=ax[1], linewidth=6, color=pm_color, linestyle='solid')
plt.tight_layout()

plt.show()